In [47]:
import pandas as pd
from tqdm.contrib.concurrent import process_map
import pathlib
import duckdb


In [48]:
with duckdb.connect('../data/protodb.db') as conn:
    df = conn.query('select * from intel.cian').df()

In [49]:
df.shape

(8032, 32)

In [52]:
df.drop_duplicates(subset=['url','price']).to_parquet('../data/sample_data.parquet')

In [53]:
sample_ = pd.read_parquet('../data/sample_data.parquet')

In [54]:
# with duckdb.connect('../data/protodb.db') as conn:
#     conn.execute('create schema if not exists intel ')
#     conn.execute('create table if not exists intel.cian as select * from sample_')
#     df = conn.query('select * from intel.cian').df()